In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
df = pd.read_csv('data/childcare-immunizations_2016-17_to_2019-20.csv')

In [4]:
df.head()

SCHOOL_YEAR  FACILITY_NUMBER             FACILITY_NAME     CITY  COUNTY  \
0   2019-2020           102814  Orange Unified Pre-K SDC  Anaheim  ORANGE   
1   2019-2020           102814  Orange Unified Pre-K SDC  Anaheim  ORANGE   
2   2019-2020           102814  Orange Unified Pre-K SDC  Anaheim  ORANGE   
3   2019-2020           102814  Orange Unified Pre-K SDC  Anaheim  ORANGE   
4   2019-2020           102814  Orange Unified Pre-K SDC  Anaheim  ORANGE   

  FACILITY_TYPE  ENROLLMENT     CATEGORY  COUNT  
0        PUBLIC         188  CONDITIONAL    1.0  
1        PUBLIC         188          DTP  157.0  
2        PUBLIC         188         HEPB  158.0  
3        PUBLIC         188          HIB  158.0  
4        PUBLIC         188          MMR  158.0

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104897 entries, 0 to 104896
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SCHOOL_YEAR      104897 non-null  object 
 1   FACILITY_NUMBER  104897 non-null  int64  
 2   FACILITY_NAME    104897 non-null  object 
 3   CITY             104897 non-null  object 
 4   COUNTY           104897 non-null  object 
 5   FACILITY_TYPE    104897 non-null  object 
 6   ENROLLMENT       104897 non-null  int64  
 7   CATEGORY         104897 non-null  object 
 8   COUNT            103177 non-null  float64
dtypes: float64(1), int64(2), object(6)
memory usage: 7.2+ MB


In [6]:
#tidy data
df_new = df.pivot(
    index = ['SCHOOL_YEAR','FACILITY_NUMBER','FACILITY_NAME','CITY','COUNTY','FACILITY_TYPE','ENROLLMENT'],
    columns = ['CATEGORY'],
    values = ['COUNT'],
).reset_index()

# rename the columns
df_new.columns = ['SCHOOL_YEAR','FACILITY_NUMBER','FACILITY_NAME','CITY','COUNTY','FACILITY_TYPE','ENROLLMENT','CONDITIONAL','DTP','HEPB','HIB','MMR','OTHERS','OVERDUE','PBE','PME','POLIO','UP-TO-DATE','VARI']

In [7]:
df_new.head()

SCHOOL_YEAR  FACILITY_NUMBER                   FACILITY_NAME           CITY  \
0   2016-2017               24         REDWOOD PARENTS NURSERY  EMERALD HILLS   
1   2016-2017               50  HERITAGE OAK CHILDREN'S CENTER       ATHERTON   
2   2016-2017           102814        ORANGE UNIFIED PRE-K SDC        ANAHEIM   
3   2016-2017           111377              WESTPARK PRESCHOOL       ROSAMOND   
4   2016-2017           115147    CANYON EARLY LEARNING CENTER       MONROVIA   

        COUNTY FACILITY_TYPE  ENROLLMENT  CONDITIONAL    DTP   HEPB   HIB  \
0    SAN MATEO       PRIVATE          61          1.0    NaN   58.0   NaN   
1    SAN MATEO        PUBLIC          30          3.0   27.0   28.0  28.0   
2       ORANGE        PUBLIC         235          0.0  225.0  218.0   NaN   
3         KERN        PUBLIC          72          1.0    NaN    NaN   NaN   
4  LOS ANGELES        PUBLIC         238          4.0  234.0    NaN   NaN   

    MMR  OTHERS  OVERDUE  PBE  PME  POLIO  UP-TO-DATE  VARI  
0  59.0     0.0      0.0  2.0  0.0    NaN        58.0   NaN  
1  28.0     0.0      0.0  0.0  0.0   28.0        27.0  28.0  
2   NaN     3.0     18.0  0.0  3.0  230.0       211.0   NaN  
3  70.0     0.0      0.0  0.0  2.0    NaN        69.0   NaN  
4   NaN     0.0      0.0  0.0  0.0    NaN       234.0   NaN

In [8]:
# question 1
df_new.shape[0]

11746

In [9]:
#question 2
df_new['ENROLLMENT'].agg(['max','min','mean','median','std','sum'])

max         1028.000000
min           20.000000
mean          73.326409
median        62.000000
std           47.912890
sum       861292.000000
Name: ENROLLMENT, dtype: float64

In [10]:
df_Q3 = pd.DataFrame(df_new.loc[df_new['SCHOOL_YEAR']=='2019-2020','FACILITY_TYPE'])

In [11]:
df_Q3.head()

FACILITY_TYPE
9076        PUBLIC
9077        PUBLIC
9078        PUBLIC
9079        PUBLIC
9080        PUBLIC

In [12]:
df_Q3_plot = alt.Chart(df_Q3,title='Facility type in 2019-2020').mark_bar().encode(x=alt.X('FACILITY_TYPE').title('Facility type'),y=alt.Y('count()'))

df_Q3_plot = df_Q3_plot.properties(width=400)

In [13]:
df_Q3_plot

alt.Chart(...)

In [14]:
pip install vl-convert-python

  Obtaining dependency information for vl-convert-python from https://files.pythonhosted.org/packages/a7/6b/48f6d47a92eaf6f0dd235146307a7eb0d179b78d2faebc53aca3f1e49177/vl_convert_python-1.8.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 50.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
df_Q3_plot.save('image/df_Q3_plot.png')

In [16]:
df_Q4 = df_new[['ENROLLMENT','FACILITY_TYPE']]
df_Q4.head()

ENROLLMENT FACILITY_TYPE
0          61       PRIVATE
1          30        PUBLIC
2         235        PUBLIC
3          72        PUBLIC
4         238        PUBLIC

In [17]:
df_Q4_plot = alt.Chart(df_Q4).mark_bar().encode(
    x=alt.X('ENROLLMENT').bin(maxbins=40).title('Enrollment'),
    y=alt.Y('count()').stack(False),
    color = alt.Color('FACILITY_TYPE:N').title('FACILITY_TYPE'))

v_line = alt.Chart(df_Q4).mark_rule(strokeDash=[6],size=1.5).encode(x='mean(ENROLLMENT)')
df_Q4_plot = df_Q4_plot + v_line

df_Q4_relative = df_Q4_plot.properties(
    height=200
).facet(
    "FACILITY_TYPE:N",
    columns=1,
    title=""
)
df_Q4_relative

alt.FacetChart(...)

In [18]:
df_Q4_relative.save('image/df_Q4_plot.png')

In [19]:
df_Q5 = df_new[['ENROLLMENT','UP-TO-DATE','FACILITY_TYPE']].copy()

In [20]:
df_Q5.head()

ENROLLMENT  UP-TO-DATE FACILITY_TYPE
0          61        58.0       PRIVATE
1          30        27.0        PUBLIC
2         235       211.0        PUBLIC
3          72        69.0        PUBLIC
4         238       234.0        PUBLIC

In [21]:
df_Q5['RATIO'] = df_Q5['UP-TO-DATE'] / df_Q5['ENROLLMENT']

In [22]:
df_Q5

ENROLLMENT  UP-TO-DATE FACILITY_TYPE     RATIO
0              61        58.0       PRIVATE  0.950820
1              30        27.0        PUBLIC  0.900000
2             235       211.0        PUBLIC  0.897872
3              72        69.0        PUBLIC  0.958333
4             238       234.0        PUBLIC  0.983193
...           ...         ...           ...       ...
11741          48        44.0        PUBLIC  0.916667
11742          66        63.0        PUBLIC  0.954545
11743          45        41.0        PUBLIC  0.911111
11744          60        58.0        PUBLIC  0.966667
11745          37        32.0       PRIVATE  0.864865

[11746 rows x 4 columns]

In [23]:
df_Q5_plot = alt.Chart(df_Q5).mark_point(size=10,opacity=0.2).encode(
    x = alt.X('ENROLLMENT'),
    y = alt.Y('RATIO')
)
df_Q5_plot

alt.Chart(...)

In [24]:
df_Q5_plot = alt.Chart(df_Q5).mark_point(size=10,opacity=0.2).encode(
    x = alt.X('ENROLLMENT').title('Enrollment'),
    y = alt.Y('RATIO').title('Enrollment/up-to-data(%)'),
    color = alt.Color('FACILITY_TYPE').title('facility type')
)
df_Q5_relative = df_Q5_plot.properties(
    height=200
).facet(
    "FACILITY_TYPE:N",
    columns=1,
    title=""
)
df_Q5_relative

alt.FacetChart(...)

In [25]:
df_Q5_plot.save('image/df_Q5_plot.png')
df_Q5_relative.save('image/df_Q5_relative.png')

In [26]:
df_Q6 = df_new[['UP-TO-DATE','ENROLLMENT','SCHOOL_YEAR','FACILITY_TYPE']].copy()
df_Q6['RATIO'] = df_Q6['UP-TO-DATE'] / df_Q6['ENROLLMENT']
df_Q6.head()

UP-TO-DATE  ENROLLMENT SCHOOL_YEAR FACILITY_TYPE     RATIO
0        58.0          61   2016-2017       PRIVATE  0.950820
1        27.0          30   2016-2017        PUBLIC  0.900000
2       211.0         235   2016-2017        PUBLIC  0.897872
3        69.0          72   2016-2017        PUBLIC  0.958333
4       234.0         238   2016-2017        PUBLIC  0.983193

In [27]:
df_Q6_1 = df_Q6.groupby(['SCHOOL_YEAR','FACILITY_TYPE'])['RATIO'].mean().reset_index()
df_Q6_1

SCHOOL_YEAR FACILITY_TYPE     RATIO
0    2016-2017    HEAD START  0.925501
1    2016-2017       PRIVATE  0.878438
2    2016-2017        PUBLIC  0.885249
3    2017-2018    HEAD START  0.939320
4    2017-2018       PRIVATE  0.881389
5    2017-2018        PUBLIC  0.889480
6    2018-2019    HEAD START  0.942280
7    2018-2019       PRIVATE  0.882590
8    2018-2019        PUBLIC  0.891125
9    2019-2020    HEAD START  0.921639
10   2019-2020       PRIVATE  0.885876
11   2019-2020        PUBLIC  0.882366

In [28]:
df_Q6_2 = df_Q6.groupby('SCHOOL_YEAR')['RATIO'].mean().reset_index()
df_Q6_2['FACILITY_TYPE'] = 'All'
df_Q6_2.head()

SCHOOL_YEAR     RATIO FACILITY_TYPE
0   2016-2017  0.883978           All
1   2017-2018  0.887446           All
2   2018-2019  0.889685           All
3   2019-2020  0.887232           All

In [29]:
import pandas as pd

df_Q6_renew = pd.concat([df_Q6_1, df_Q6_2])
df_Q6_renew

SCHOOL_YEAR FACILITY_TYPE     RATIO
0    2016-2017    HEAD START  0.925501
1    2016-2017       PRIVATE  0.878438
2    2016-2017        PUBLIC  0.885249
3    2017-2018    HEAD START  0.939320
4    2017-2018       PRIVATE  0.881389
5    2017-2018        PUBLIC  0.889480
6    2018-2019    HEAD START  0.942280
7    2018-2019       PRIVATE  0.882590
8    2018-2019        PUBLIC  0.891125
9    2019-2020    HEAD START  0.921639
10   2019-2020       PRIVATE  0.885876
11   2019-2020        PUBLIC  0.882366
0    2016-2017           All  0.883978
1    2017-2018           All  0.887446
2    2018-2019           All  0.889685
3    2019-2020           All  0.887232

In [30]:
df_Q6_answer = alt.Chart(df_Q6_renew).mark_line().encode(
    x = alt.X('SCHOOL_YEAR').title('school year'),
    y = alt.Y('RATIO',scale=alt.Scale(domain=[0.8,1])).title('ratio for up-to-data'),
    color = alt.Color('FACILITY_TYPE')
).properties(
    height = 300,
    width = 300
)
df_Q6_answer

alt.Chart(...)

In [31]:
df_Q6_answer.save('image/df_Q6_plot.png')